# Chainer cifar10 tutorial
自分の理解が深まるように少し改変  
https://github.com/pfnet/chainer/blob/master/examples/cifar/train_cifar.py

In [1]:
#!/usr/bin/env python
from __future__ import print_function
import argparse
import sys; sys.argv=['']; del sys

import numpy as np
import chainer
import chainer.functions as F
import chainer.links as L
from chainer import training
from chainer.training import extensions
from chainer import Variable
import copy # test iterator copy


In [2]:
dataset = 'cifar10'
# dataset = 'mnist'

batchsize = 128
epochsize = 300

# Load dataset

In [3]:
if dataset == 'cifar10':
    print('Using CIFAR10 dataset.')
    class_labels = 10
    train, test = chainer.datasets.get_cifar10()
elif dataset == 'cifar100':
    print('Using CIFAR100 dataset.')
    class_labels = 100
    train, test = chainer.datasets.get_cifar100()
elif dataset == 'mnist':
    print('Using mnist dataset.')
    class_labels = 10
    train, test = chainer.datasets.get_mnist()
    train = [(it[0].reshape(1, 28, 28),it[1]) for it in train]
    test = [(it[0].reshape(1, 28, 28),it[1]) for it in test]

Using CIFAR10 dataset.


In [4]:
print(len(train))
print(len(test))

50000
10000


In [5]:
train[0][0].shape

(3, 32, 32)

In [6]:
type(train[0][0])

numpy.ndarray

In [7]:
# train = [(it[0].flatten(),it[1]) for it in train]
# test = [(it[0].flatten(),it[1]) for it in test]

In [8]:
# Load the cifar dataset
train_iter = chainer.iterators.SerialIterator(train, batchsize)
test_iter = chainer.iterators.SerialIterator(test, batchsize, repeat=False, shuffle=False)
testsize = len(test)
testsize

10000

# Network

In [9]:
class VGG(chainer.Chain):
    def __init__(self, class_labels):
        initializer = chainer.initializers.HeNormal()
        c1 = 32
        c2 = 64
        c3 = 64
        c4 = 128

        super(VGG, self).__init__(
             # the size of the inputs to each layer will be inferred
            conv1=L.Convolution2D(None, c1, 3, stride=1, pad=0),
            conv2=L.Convolution2D(32, c2, 3, stride=1, pad=0),
            conv3=L.Convolution2D(64, c3, 3, stride=1, pad=1),
            conv4=L.Convolution2D(64, c4, 3, stride=1, pad=1),
            fc1=L.Linear(512, 1000, initialW=initializer),
            fc2=L.Linear(1000, class_labels, initialW=initializer),
            bnorm1=L.BatchNormalization(c1),
            bnorm2=L.BatchNormalization(c2),
            bnorm3=L.BatchNormalization(c3),
            bnorm4=L.BatchNormalization(c4),
        )

    def __call__(self, x):
        h = F.max_pooling_2d(F.relu(self.bnorm1(self.conv1(x))), 2, stride=2)
        h = F.max_pooling_2d(F.relu(self.bnorm2(self.conv2(h))), 2, stride=2)
        h = F.max_pooling_2d(F.relu(self.bnorm3(self.conv3(h))), 2, stride=2)
        h = F.max_pooling_2d(F.relu(self.bnorm4(self.conv4(h))), 2, stride=2)
        h = F.relu(self.fc1(h))
        y = self.fc2(h)        
        return y

In [10]:
class Classifier(chainer.Chain):
    def __init__(self, predictor):
        super(Classifier, self).__init__(predictor=predictor)
        
    def clear(self):
        self.loss = None
        self.accuracy = None
        
    def __call__(self, x, t):
        self.clear()
        y = self.predictor(x)
        loss = F.softmax_cross_entropy(y, t)
        self.accuracy = F.accuracy(y, t)
        chainer.report({'loss': loss, 'accuracy': self.accuracy}, self)
        return loss


In [11]:

gpu_id = 0 #  use gpu
model = Classifier(VGG(class_labels))

if gpu_id >= 0:
    chainer.cuda.get_device(gpu_id).use()  # Make a specified GPU current
    model.to_gpu()  # Copy the model to the GPU

# Setup an optimizer

In [12]:
optimizer = chainer.optimizers.MomentumSGD(0.1)
optimizer.use_cleargrads()
optimizer.setup(model)
optimizer.add_hook(chainer.optimizer.WeightDecay(5e-4))

In [13]:
batch = train_iter.next()

In [14]:
xp = model.xp
xp

<module 'cupy' from '/home/komatsu/anaconda2/lib/python2.7/site-packages/cupy/__init__.pyc'>

In [15]:
x = xp.asarray([it[0] for it in batch], dtype=np.float32)
t = xp.asarray([it[1] for it in batch], dtype=np.int32)

In [16]:
x[0].shape

(3, 32, 32)

In [17]:
# predict without Classifier
y = model.predictor(x)
loss = F.softmax_cross_entropy(y, t)
#print(y.data)
print(loss.data)

2.79799675941


In [18]:
y.data

array([[-2.77061367,  0.50353652,  0.16753215, ...,  0.20824817,
        -1.42984438, -0.35557461],
       [-2.36415195,  0.38795841,  1.51521218, ...,  1.03411067,
        -0.69066375,  0.36017844],
       [-2.6544559 ,  0.22267927,  0.05753254, ...,  0.63379568,
        -0.14436273,  0.09109555],
       ..., 
       [-1.90264261, -1.01524186,  2.96824145, ..., -0.40632311,
        -0.76937735,  0.14880049],
       [-2.24681449, -0.38022453, -1.4336077 , ..., -0.42803025,
        -1.74473071, -0.41486323],
       [-1.83196187, -1.50650048,  0.46756282, ..., -0.24303272,
        -1.77973819,  0.38863242]], dtype=float32)

In [ ]:
t[0]

array(5, dtype=int32)

In [ ]:
%%time
from tqdm import tqdm_notebook as tqdm
# from tqdm import tqdm

# run
xp = model.xp

pbar = tqdm(xrange(epochsize))
for epoch in pbar:
    for batch in train_iter:
        # data separation
        x = xp.asarray([it[0] for it in batch], dtype=np.float32)
        t = xp.asarray([it[1] for it in batch], dtype=np.int32)
        # compute grad
        loss = model(x, t)
        model.cleargrads()
        loss.backward()
        optimizer.update()

        # terminate
        if train_iter.is_new_epoch is True:
            break

    # evaluate model   
    sum_loss = 0
    sum_acc = 0
    test_iter_copy = copy.copy(test_iter)
    for test_batch in test_iter_copy:
        # data separation
        x = xp.asarray([it[0] for it in test_batch], dtype=np.float32)
        t = xp.asarray([it[1] for it in test_batch], dtype=np.int32)
        # compute grad
        loss = model(x, t)
        sum_loss += loss.data * len(test_batch)
        sum_acc += model.accuracy.data * len(test_batch)
    mean_loss = sum_loss / testsize
    mean_acc = sum_acc / testsize
#     print(''.format(epoch=epoch))
    print('epoch : {epoch}, Mean loss: {loss}, Mean accuracy: {acc}'.format(epoch=epoch, loss=mean_loss, acc=mean_acc))
    # pbar.set_description('epoch : {epoch}'.format(epoch=epoch))

epoch : 0, Mean loss: 1.30909144878, Mean accuracy: 0.5257999897
epoch : 1, Mean loss: 1.01005339622, Mean accuracy: 0.640600025654
epoch : 2, Mean loss: 0.908256649971, Mean accuracy: 0.678600013256
epoch : 3, Mean loss: 0.881660282612, Mean accuracy: 0.691900014877
epoch : 4, Mean loss: 0.885430455208, Mean accuracy: 0.692200005054
epoch : 5, Mean loss: 0.798857390881, Mean accuracy: 0.725499987602
epoch : 6, Mean loss: 0.788113892078, Mean accuracy: 0.729799985886
epoch : 7, Mean loss: 0.729344546795, Mean accuracy: 0.743399977684
epoch : 8, Mean loss: 0.759082555771, Mean accuracy: 0.739300012589
epoch : 9, Mean loss: 0.692937910557, Mean accuracy: 0.764199972153
epoch : 10, Mean loss: 0.679129242897, Mean accuracy: 0.767499983311
epoch : 11, Mean loss: 0.692488789558, Mean accuracy: 0.765100002289
epoch : 12, Mean loss: 0.72065281868, Mean accuracy: 0.749199986458
epoch : 13, Mean loss: 0.718389153481, Mean accuracy: 0.763700008392
epoch : 14, Mean loss: 0.68884575367, Mean accura